<a href="https://colab.research.google.com/github/susanta-pal/NLP/blob/main/Assignment_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from transformers import AutoTokenizer, BertModel
from torch.utils.data import TensorDataset, DataLoader
import sys
import torch
import numpy as np



if __name__ == "__main__":
    model = BertModel.from_pretrained("bert-base-uncased")
    input_string = input("Enter two sentences separated by a comma:\n")

    sentence1, sentence2 = input_string.split(",", 1)
    sentences = [sentence1, sentence2]

    # Tokenize each sentence
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
    sentences_tokenized = [tokenizer.tokenize(s) for s in sentences]
    MAX_LENGTH = max([len(sentences_tokenized[0]), len(sentences_tokenized[1])])

    # Generate IDs of each token and add padding to sentences smaller than given threshold
    ids = [tokenizer.convert_tokens_to_ids(t) for t in sentences_tokenized]
    # Pad the tokens
    ids = np.asarray([np.pad(i, (0, MAX_LENGTH-len(i)), mode='constant') for i in ids])


    # Generate the masks
    amasks = np.asarray([[float(i>0) for i in seq] for seq in ids])

    # Get the output from the model
    output = model.forward(torch.tensor(ids), torch.tensor(amasks))

    # Now create two vector representations for each sentence
    # pool_vectors (2x768) is the vector generated by perofrming max-pooling over the hidden states
    #pool_vectors = --------------      # Fill up this '----------' section
    pool_vectors = torch.max(output.last_hidden_state, dim=1).values
    # cls_vectors (2x768) is the vector generated by taking the CLS token
    #cls_vectors = --------------       # Fill up this '----------' section
    cls_vectors = output.last_hidden_state[:, 0, :]


    # Compute the cosine similarity
    #cosine_pooling = --------------       # Fill up this '----------' section
    cosine_pooling = torch.nn.functional.cosine_similarity(pool_vectors[0], pool_vectors[1], dim=0)
    cosine_cls = torch.nn.functional.cosine_similarity(cls_vectors[0], cls_vectors[1], dim=0)
    #cosine_cls = ------------------       # Fill up this '----------' section

    # Finally print out the values
    print(np.round(cosine_pooling.item(), 2), np.round(cosine_cls.item(), 2))

Enter two sentences separated by a comma:
The restaurant offers a variety of vegan dishes , There are many plant-based options on the menu at the cafe
0.64 0.36
